In [1]:
''' Point Addition '''

# Algorithm for the addition operation on an elliptic curve of the form y^2 = ax^3 + bx + c. Note that only a determines the addition formulas.
# We require that 4a^3 + 27b^2 != 0. We'll assume a=1 and b=0 by default. 

def ecadd(P, Q, a=1, p=9739): 
    if P == 'O':
        return Q
    if Q == 'O':
        return P
    
    x1, y1, x2, y2 = P[0], P[1], Q[0], Q[1]

    if x1 == x2 and y1 == -y2:
        return 'O'
    
    if P != Q:
        lam = (y2 - y1) * pow(x2 - x1, -1, p) % p
    else:
        lam = (3*x1**2 + a) * pow(2*y1, -1, p) % p

    x3 = (lam**2 - x1 - x2) % p
    y3 = (lam * (x1 - x3) - y1) % p

    return x3, y3 
    

In [18]:
X = (5274, 2841)
Y = (8669, 740)
a = 497 

print(ecadd(X, Y, a))
print(ecadd(X, X, a))

(1024, 4440)
(7284, 2107)


In [44]:
P = (493, 5564)
Q = (1539, 4742)
R = (4403,5202)

# Flag
(x, y) = ecadd(ecadd(ecadd(P , P, a), Q, a), R, a) # P + P + Q + R
x, y

(4215, 2162)

In [9]:
''' Scalar Multiplication '''

def scalar(P, a=1, n=1):
    Q, R = P, 'O'

    while n > 0:
        if n % 2 == 1:
            R = ecadd(R, Q, a)
        Q = ecadd(Q, Q, a)
        n = n // 2 # if n is odd this is (n-1)/2 
    
    return R

In [13]:
# Test cases
X = (5323, 5438)
P = (2339, 2213)
print(scalar(X, 497))
print(scalar(X, 497, 2))
print(scalar(X, 497, 1337)) # This checks out. 
print(scalar(P, 497, 7863))

(5323, 5438)
(1512, 8728)
(1089, 6931)
(9467, 2742)


In [21]:
''' Curves and Logs '''

Qa = (815, 3190) # a point on the curve E(F_p) where E is defined by y^2 = x^3 + 497x + 1768, p = 9739 and G = (1804,5368) (not secure!!). 
nB = 1829

shared_secret = scalar(Qa, 497, nB)
print(shared_secret)

# Now we want to take the SHA1 of the x coordinate of the shared_secret, namely x = 7929. 
x = 7929 

# SHA1 hash, from website: 
hash = 0x7e80e9d57ca4aed68e2599b37fef818dc4dab502
int(hash).to_bytes(25, 'big')

(7929, 707)


b'\x00\x00\x00\x00\x00~\x80\xe9\xd5|\xa4\xae\xd6\x8e%\x99\xb3\x7f\xef\x81\x8d\xc4\xda\xb5\x02'

In [35]:
''' Efficient exchange '''

# So we get Alice's x coordinate for the secret key. We can calculate the y from y^2 = x^3 + 497x + 1768, mod p = 9739 (congruent to 3 mod 4). 
# Basically just calculate the right hand side (mod p) and then take the square root mod p, which is just result**(p-1)/4. 
p = 9739
x = 4726
nB = 6534 

result = (pow(x, 3, p) + 497*x + 1768) % p
y = pow(result, (p+1)//4, p) 
y2 = p - y # the other root 

# Now that we have Qa = (x, y) we calculate nB * Qa
Q = (x, y) 
Q2 = (x, y2)

## Possible keys. 
key = scalar(Q, 497, nB)
key2 = scalar(Q2, 497, nB) 
print(f'Possible keys: {key, key2}') # then we use this to decrypt the flag in the next code block. 

info = {'iv': 'cd9da9f1c60925922377ea952afc212c', 'encrypted_flag': 'febcbe3a3414a730b125931dccf912d2239f3e969c4334d95ed0ec86f6449ad8'}

Possible keys: ((1791, 2181), (1791, 7558))


In [37]:
from Crypto.Cipher import AES
from Crypto.Util.Padding import pad, unpad
import hashlib


def is_pkcs7_padded(message):
    padding = message[-message[-1]:]
    return all(padding[i] == len(padding) for i in range(0, len(padding)))


def decrypt_flag(shared_secret: int, iv: str, ciphertext: str):
    # Derive AES key from shared secret
    sha1 = hashlib.sha1()
    sha1.update(str(shared_secret).encode('ascii'))
    key = sha1.digest()[:16]
    # Decrypt flag
    ciphertext = bytes.fromhex(ciphertext)
    iv = bytes.fromhex(iv)
    cipher = AES.new(key, AES.MODE_CBC, iv)
    plaintext = cipher.decrypt(ciphertext)

    if is_pkcs7_padded(plaintext):
        return unpad(plaintext, 16).decode('ascii')
    else:
        return plaintext.decode('ascii')


shared_secret = 1791 # from the previous block 
iv = 'cd9da9f1c60925922377ea952afc212c'
ciphertext = 'febcbe3a3414a730b125931dccf912d2239f3e969c4334d95ed0ec86f6449ad8'

print(decrypt_flag(shared_secret, iv, ciphertext))

crypto{3ff1c1ent_k3y_3xch4ng3}
